日志分析需要具备哪些步骤？
- 读取
- 提取（提取有用信息，组成结构化数据）
- 分析
- 保存数据结构

In [3]:
line = '66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "GET /robots.txt HTTP/1.1" 404 162 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"'

- ip
- 时间
- 请求方式、请求地址
- 状态码
- 大小
- user-agent

In [4]:
tmp = []
ret = []
for c in line:
    if c == ' ':
        ret.append(''.join(tmp))
        tmp.clear()
    else:
        tmp.append(c)

In [5]:
ret

['66.249.69.131',
 '-',
 '-',
 '[10/Aug/2016:03:20:09',
 '+0800]',
 '"GET',
 '/robots.txt',
 'HTTP/1.1"',
 '404',
 '162',
 '"-"',
 '"Mozilla/5.0',
 '(compatible;',
 'Googlebot/2.1;']

In [6]:
tmp = []
ret = []
split = True
for c in line:
    if c == '[':
        split = False
        continue
    if c == ']':
        split = True
        continue
    if c == ' ' and split:
        ret.append(''.join(tmp))
        tmp.clear()
    else:
        tmp.append(c)

In [7]:
ret

['66.249.69.131',
 '-',
 '-',
 '10/Aug/2016:03:20:09 +0800',
 '"GET',
 '/robots.txt',
 'HTTP/1.1"',
 '404',
 '162',
 '"-"',
 '"Mozilla/5.0',
 '(compatible;',
 'Googlebot/2.1;']

In [8]:
tmp = []
ret = []
split = True
for c in line:
    if c == '[':
        split = False
        continue
    if c == ']':
        split = True
        continue
        
    if c == '"':
        split = not split
        continue
        
    if c == ' ' and split:
        ret.append(''.join(tmp))
        tmp.clear()
    else:
        tmp.append(c)

In [10]:
ret

['66.249.69.131',
 '-',
 '-',
 '10/Aug/2016:03:20:09 +0800',
 'GET /robots.txt HTTP/1.1',
 '404',
 '162',
 '-']

In [11]:
tmp

['M',
 'o',
 'z',
 'i',
 'l',
 'l',
 'a',
 '/',
 '5',
 '.',
 '0',
 ' ',
 '(',
 'c',
 'o',
 'm',
 'p',
 'a',
 't',
 'i',
 'b',
 'l',
 'e',
 ';',
 ' ',
 'G',
 'o',
 'o',
 'g',
 'l',
 'e',
 'b',
 'o',
 't',
 '/',
 '2',
 '.',
 '1',
 ';',
 ' ',
 '+',
 'h',
 't',
 't',
 'p',
 ':',
 '/',
 '/',
 'w',
 'w',
 'w',
 '.',
 'g',
 'o',
 'o',
 'g',
 'l',
 'e',
 '.',
 'c',
 'o',
 'm',
 '/',
 'b',
 'o',
 't',
 '.',
 'h',
 't',
 'm',
 'l',
 ')']

In [12]:
tmp = []
ret = []
split = True
for c in line:
    if c == '[':
        split = False
        continue
    if c == ']':
        split = True
        continue
        
    if c == '"':
        split = not split
        continue
        
    if c == ' ' and split:
        ret.append(''.join(tmp))
        tmp.clear()
    else:
        tmp.append(c)
ret.append(''.join(tmp))

In [13]:
ret

['66.249.69.131',
 '-',
 '-',
 '10/Aug/2016:03:20:09 +0800',
 'GET /robots.txt HTTP/1.1',
 '404',
 '162',
 '-',
 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)']

In [18]:
def extract(line):
    names = ('remote', '', '', 'time', 'request', 'status', 'length', '', 'ua')
    tmp = []
    ret = []
    split = True
    for c in line:
        if c == '[':
            split = False
            continue
        if c == ']':
            split = True
            continue

        if c == '"':
            split = not split
            continue

        if c == ' ' and split:
            ret.append(''.join(tmp))
            tmp.clear()
        else:
            tmp.append(c)
    ret.append(''.join(tmp))
    result = dict(zip(names, ret))
    result.pop('', None)
    return result

In [19]:
extract(line)

{'length': '162',
 'remote': '66.249.69.131',
 'request': 'GET /robots.txt HTTP/1.1',
 'status': '404',
 'time': '10/Aug/2016:03:20:09 +0800',
 'ua': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}

In [20]:
import datetime

In [21]:
def strptime(src: str) -> datetime.datetime:
    return datetime.datetime.strptime(src, '%d/%b/%Y:%H:%M:%S %z')

In [22]:
strptime('10/Aug/2016:03:20:09 +0800')

datetime.datetime(2016, 8, 10, 3, 20, 9, tzinfo=datetime.timezone(datetime.timedelta(0, 28800)))

In [23]:
def extract(line):
    names = ('remote', '', '', 'time', 'request', 'status', 'length', '', 'ua')
    tmp = []
    ret = []
    split = True
    for c in line:
        if c == '[':
            split = False
            continue
        if c == ']':
            split = True
            continue

        if c == '"':
            split = not split
            continue

        if c == ' ' and split:
            ret.append(''.join(tmp))
            tmp.clear()
        else:
            tmp.append(c)
    ret.append(''.join(tmp))
    result = dict(zip(names, ret))
    result.pop('', None)
    
    result['length'] = int(result['length'])
    result['status'] = int(result['status'])
    result['time'] = strptime(result['time'])
    return result

In [24]:
extract(line)

{'length': 162,
 'remote': '66.249.69.131',
 'request': 'GET /robots.txt HTTP/1.1',
 'status': 404,
 'time': datetime.datetime(2016, 8, 10, 3, 20, 9, tzinfo=datetime.timezone(datetime.timedelta(0, 28800))),
 'ua': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}

In [25]:
from collections import namedtuple

In [26]:
Request = namedtuple('Request', ['method', 'url', 'version'])

In [29]:
def request(src: str) -> Request:
    return Request(*src.split())

In [30]:
request('GET /robots.txt HTTP/1.1')

Request(method='GET', url='/robots.txt', version='HTTP/1.1')

In [31]:
def extract(line):
    names = ('remote', '', '', 'time', 'request', 'status', 'length', '', 'ua')
    tmp = []
    ret = []
    split = True
    for c in line:
        if c == '[':
            split = False
            continue
        if c == ']':
            split = True
            continue

        if c == '"':
            split = not split
            continue

        if c == ' ' and split:
            ret.append(''.join(tmp))
            tmp.clear()
        else:
            tmp.append(c)
    ret.append(''.join(tmp))
    result = dict(zip(names, ret))
    result.pop('', None)
    
    result['length'] = int(result['length'])
    result['status'] = int(result['status'])
    result['time'] = strptime(result['time'])
    result['request'] = request(result['request'])
    return result

In [32]:
extract(line)

{'length': 162,
 'remote': '66.249.69.131',
 'request': Request(method='GET', url='/robots.txt', version='HTTP/1.1'),
 'status': 404,
 'time': datetime.datetime(2016, 8, 10, 3, 20, 9, tzinfo=datetime.timezone(datetime.timedelta(0, 28800))),
 'ua': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}

In [33]:
mapping = {
    'remote': lambda x: x,
    'time': lambda x: datetime.datetime.strptime(x, '%d/%b/%Y:%H:%M:%S %z'),
    'request': lambda x: Request(*x.split()),
    'status': int,
    'length': int,
    'ua': lambda x: x
}

In [41]:
MapItem = namedtuple('MapItem', ['name', 'convert'])

mapping = [
    MapItem('remote', lambda x: x),
    MapItem('', None),
    MapItem('', None),
    MapItem('time', lambda x: datetime.datetime.strptime(x, '%d/%b/%Y:%H:%M:%S %z')),
    MapItem('request', lambda x: Request(*x.split())),
    MapItem('status', int),
    MapItem('length', int),
    MapItem('', None),
    MapItem('ua', lambda x: x)
]

def extract(line):

    tmp = []
    ret = []
    split = True
    for c in line:
        if c == '[':
            split = False
            continue
        if c == ']':
            split = True
            continue

        if c == '"':
            split = not split
            continue

        if c == ' ' and split:
            ret.append(''.join(tmp))
            tmp.clear()
        else:
            tmp.append(c)
    ret.append(''.join(tmp))

    result = dict()
    for i, item in enumerate(mapping):
        if item.name:
            result[item.name] = item.convert(ret[i])
    return result

In [42]:
extract(line)

{'length': 162,
 'remote': '66.249.69.131',
 'request': Request(method='GET', url='/robots.txt', version='HTTP/1.1'),
 'status': 404,
 'time': datetime.datetime(2016, 8, 10, 3, 20, 9, tzinfo=datetime.timezone(datetime.timedelta(0, 28800))),
 'ua': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}

### 正则表达式

BRE，ERE，PCRE以及PCRE 的其他方言:
- grep, sed 用的是BRE
- awk 用的就是ERE
- BRE 就是基础的正则表达式， 功能最弱
- ERE 扩展正则表达式，功能比BRE强
- PCRE， perl 兼容的正则表示，功能最强的

python 中 用的是 SRE， PCRE的 一个子集

In [43]:
import re

In [44]:
re.match
re.search
re.findall

<function re.findall>

In [49]:
regex = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3} - - .*'

In [48]:
re.match(regex, '66.249.69.131')

<_sre.SRE_Match object; span=(0, 13), match='66.249.69.131'>

In [51]:
re.match(regex, line )

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [52]:
line

'66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "GET /robots.txt HTTP/1.1" 404 162 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"'

In [55]:
regex = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3} - - \[.*\] ".*" .*'
re.match(regex, line )

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [56]:
regex = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3} - - \[.*\] ".*" \d+ \d+ .*'
re.match(regex, line )

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [57]:
regex = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3} - - \[.*\] ".*" \d+ \d+ ".*".*'
re.match(regex, line )

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [58]:
regex = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3} - - \[.*\] ".*" \d+ \d+ ".*" ".*"'
re.match(regex, line )

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [60]:
regex = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(.*)\] "(.*)" (\d+) (\d+) ".*" "(.*)"' # 捕获
re.match(regex, line )

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [61]:
regex = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(.*)\] "(.*)" (\d+) (\d+) ".*" "(.*)"' # 捕获
m = re.match(regex, line)

In [62]:
m.groups()

('66.249.69.131',
 '10/Aug/2016:03:20:09 +0800',
 'GET /robots.txt HTTP/1.1',
 '404',
 '162',
 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)')

In [66]:
regex = r'(?P<remote>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(?P<time>.*)\] "(?P<request>.*)" (?P<status>\d+) (?P<length>\d+) ".*" "(?P<ua>.*)"' # 捕获
m = re.match(regex, line)

In [67]:
m

<_sre.SRE_Match object; span=(0, 160), match='66.249.69.131 - - [10/Aug/2016:03:20:09 +0800] "G>

In [68]:
m.groupdict()

{'length': '162',
 'remote': '66.249.69.131',
 'request': 'GET /robots.txt HTTP/1.1',
 'status': '404',
 'time': '10/Aug/2016:03:20:09 +0800',
 'ua': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}

In [70]:
ex

In [72]:

# result= {}
# for k, v in ret.items():
#     result[k] = mapping.get(k, lambda x: x)(v)
# return result

In [73]:
matcher = re.compile(r'(?P<remote>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(?P<time>.*)\] "(?P<request>.*)" (?P<status>\d+) (?P<length>\d+) ".*" "(?P<ua>.*)"') # 捕获

In [74]:
def extract(line):
    m = matcher.match(line)
    if m:
        ret = m.groupdict()
        return {k: mapping.get(k, lambda x: x)(v) for k, v in ret.items()}
    raise Exception(line)

In [75]:
extract(line)

{'length': 162,
 'remote': '66.249.69.131',
 'request': Request(method='GET', url='/robots.txt', version='HTTP/1.1'),
 'status': 404,
 'time': datetime.datetime(2016, 8, 10, 3, 20, 9, tzinfo=datetime.timezone(datetime.timedelta(0, 28800))),
 'ua': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}

In [76]:
text = 'abcdef'

In [77]:
re.match('a', text) # 从头开始匹配 re.search('^b')

<_sre.SRE_Match object; span=(0, 1), match='a'>

In [78]:
re.match('b', text)

In [79]:
re.search('a', text) # 从任意位置开始匹配

<_sre.SRE_Match object; span=(0, 1), match='a'>

In [83]:
re.search('b', text)

<_sre.SRE_Match object; span=(1, 2), match='b'>

In [84]:
text = 'abcabc'

In [85]:
re.findall('a', text)

['a', 'a']

In [87]:
re.findall('a.*c', text)

['abcabc']

In [88]:
text = 'abc abc'

In [89]:
re.findall('a.*c', text)

['abc abc']

In [90]:
re.findall('a.\S', text)

['abc', 'abc']

In [91]:
re.finditer('a.\S', text)

In [92]:
help(re.match)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a match object, or None if no match was found.



In [93]:
text = 'Abc'

In [95]:
re.match('a', text, re.IGNORECASE)

<_sre.SRE_Match object; span=(0, 1), match='A'>

In [96]:
def load(path):
    with open('./access.log') as f:
        try:
            yield extract(f.readline())
        except:
            pass

### 滑动窗口

- size，width 决定了 每一次 聚合多少数据
- interval 多久 滑动一次

In [97]:
def window(interval, width):
    pass

In [98]:
# src: 接收源，是一个queue
# handler： 是一个函数，用于处理数据
# notify 是一个函数 用来当窗口移动的时候，通知handler
def window(src, handler, notify, interval, width):
    pass

In [99]:
Handler = namedtuple('Handler', ['handler', 'notify'])
import queue

In [101]:
def window(source: queue.Queue, handler, interval:int, width:int):
    pass

In [102]:
import datetime

In [103]:
def window(source: queue.Queue, handler, interval:int, width:int):
    store = []
    
    while True:
        data = source.get_nowait()
        
        start = datetime.datetime.now()
        
        if data:
            store.append(data)
            current = data['time']
        
        # 窗口什么时候后移？
        if (current - start).total_seconds() >= interval:
            # TODO 3件事情
            start = current
            
            handler(store)
            
            # 窗口如何后移？
            dt = current - datetime.timedelta(seconds=width)
            store = [x for x in store if x['time'] > dt]
            
        

In [104]:
q = queue.Queue()

In [105]:
def source(q):
    while True:
        q.put({'time': datetime.datetiem.now(), 'value': random.randint(0, 100)})
        time.sleep(1)

In [120]:
import threading
import random
import time

In [107]:
t = threading.Thread(target=window, args=(q))

In [116]:
import datetime
import random
import time

def window(source, handler, interval:int, width:int):
    store = []
    start = datetime.datetime.now()
    
    while True:
        data = next(source)
        current = datetime.datetime.now()
        if data:
            store.append(data)
            current = data['time']
        
        # 窗口什么时候后移？
        if (current - start).total_seconds() >= interval:
            # TODO 3件事情
            start = current
            handler(store)
            # 窗口如何后移？
            dt = current - datetime.timedelta(seconds=width)
            store = [x for x in store if x['time'] > dt]
            
def source():
    while True:
        yield {'time': datetime.datetime.now(), 'value': random.randint(0, 100)}
        time.sleep(1)
        
def handler(items):
    values = [x['value'] for x in items]
    print(sum(values) / len(values))

In [121]:
s = source()

In [122]:
next(s)

{'time': datetime.datetime(2017, 10, 14, 17, 12, 29, 668645), 'value': 70}

In [123]:
next(s)

{'time': datetime.datetime(2017, 10, 14, 17, 12, 39, 655314), 'value': 17}

In [124]:
items = []
for x in range(10):
    items.append(next(s))
    


In [125]:
items

[{'time': datetime.datetime(2017, 10, 14, 17, 13, 31, 221527), 'value': 41},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 32, 224848), 'value': 6},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 33, 229866), 'value': 10},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 34, 235049), 'value': 14},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 35, 238459), 'value': 29},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 36, 242419), 'value': 26},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 37, 246915), 'value': 72},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 38, 247316), 'value': 49},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 39, 248550), 'value': 31},
 {'time': datetime.datetime(2017, 10, 14, 17, 13, 40, 253028), 'value': 17}]

In [126]:
handler(items)

29.5


In [128]:
window(s, handler, 10, 5)

65.1
65.13333333333334
46.93333333333333


KeyboardInterrupt: 

数据加载 -> 提取 -> 分发 -> window -> 分析器

In [129]:
def despatcher(source):
    analyers = []
    
    def register(handler, interval, width):
        analyers.append(window(source, handler, interval, width))

In [130]:
import datetime
import random
import time

def window(source, handler, interval:int, width:int):
    store = []
    start = datetime.datetime.now()
    
    while True:
        # data = next(source)
        # data = source()
        data = source.get()
        current = datetime.datetime.now()
        if data:
            store.append(data)
            current = data['time']
        
        # 窗口什么时候后移？
        if (current - start).total_seconds() >= interval:
            # TODO 3件事情
            start = current
            handler(store)
            # 窗口如何后移？
            dt = current - datetime.timedelta(seconds=width)
            store = [x for x in store if x['time'] > dt]
            
def source():
    while True:
        yield {'time': datetime.datetime.now(), 'value': random.randint(0, 100)}
        time.sleep(1)
        
def handler(items):
    values = [x['value'] for x in items]
    print(sum(values) / len(values))

生产者，消费者模型

worker -> consumer

In [131]:
def despatcher(source):
    analyers = []
    
    queues = []
    
    def register(handler, interval, width):
        
        q = queue.Queue()
        queues.append(q)
        t = threading.Thread(target=window, args=(q, handler, interval, width))
        
        # analyers.append(window(source, handler, interval, width))
        analyers.append(t)
        
    def start():
        for t in analyers:
            t.start()
        for item in source:
            print(item)
            for q in queues:
                q.put(item)
                
    return register, start

In [132]:
s = source()

In [133]:
s = source()
register, start = despatcher(s)
register(handler, 5, 10)

In [134]:
register(handler, 5, 10)

In [135]:
start()

{'time': datetime.datetime(2017, 10, 14, 17, 33, 40, 812415), 'value': 20}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 41, 816037), 'value': 61}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 42, 819729), 'value': 81}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 43, 822929), 'value': 67}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 44, 823114), 'value': 39}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 45, 825893), 'value': 48}52.666666666666664

{'time': datetime.datetime(2017, 10, 14, 17, 33, 46, 829899), 'value': 22}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 47, 834181), 'value': 8}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 48, 838285), 'value': 33}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 49, 839687), 'value': 37}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 50, 839993), 'value': 63}43.54545454545455

{'time': datetime.datetime(2017, 10, 14, 17, 33, 51, 845205), 'value': 1}
{'time': datetime.datetime(2017, 10, 14, 17, 33, 52, 848281), 'va

KeyboardInterrupt: 

In [138]:
def despatcher(source):
    analyers = []
    
    queues = []
    
    def _source(q):
        while True:
            yield q.get()
    
    def register(handler, interval, width):
        
        q = queue.Queue()
        queues.append(q)
        t = threading.Thread(target=window, args=(_source(q), handler, interval, width))
        
        # analyers.append(window(source, handler, interval, width))
        analyers.append(t)
        
    def start():
        for t in analyers:
            t.start()
        for item in source:
            print(item)
            for q in queues:
                q.put(item)
                
    return register, start

In [137]:
import datetime
import random
import time

def window(source, handler, interval:int, width:int):
    store = []
    start = datetime.datetime.now()
    
    while True:
        data = next(source)
        # data = source()
        # data = source.get()
        current = datetime.datetime.now()
        if data:
            store.append(data)
            current = data['time']
        
        # 窗口什么时候后移？
        if (current - start).total_seconds() >= interval:
            # TODO 3件事情
            start = current
            handler(store)
            # 窗口如何后移？
            dt = current - datetime.timedelta(seconds=width)
            store = [x for x in store if x['time'] > dt]
            
def source():
    while True:
        yield {'time': datetime.datetime.now(), 'value': random.randint(0, 100)}
        time.sleep(1)
        
def handler(items):
    values = [x['value'] for x in items]
    print(sum(values) / len(values))

In [139]:
s = source()
register, start = despatcher(s)
register(handler, 5, 10)

In [140]:
start()

{'time': datetime.datetime(2017, 10, 14, 17, 36, 37, 745709), 'value': 17}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 38, 746512), 'value': 56}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 39, 747298), 'value': 10}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 40, 751994), 'value': 41}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 41, 752385), 'value': 99}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 42, 752803), 'value': 78}50.166666666666664

{'time': datetime.datetime(2017, 10, 14, 17, 36, 43, 753847), 'value': 18}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 44, 758957), 'value': 80}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 45, 763340), 'value': 75}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 46, 767371), 'value': 5}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 47, 770975), 'value': 10}44.45454545454545

{'time': datetime.datetime(2017, 10, 14, 17, 36, 48, 775778), 'value': 97}
{'time': datetime.datetime(2017, 10, 14, 17, 36, 49, 780678), 'v

KeyboardInterrupt: 